# Data

In [1]:
# Basic
import torch
import math
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np
from numpy import random
torch.set_num_threads(4)

# PDEi
from utils_pde.utils_pde_2dpoisson import Poisson2D

# Viz
from utils_tools.utils_result_viz import plot_predictions_2D

# Base Mdoels
from utils_uqmd.utils_uq_dropout import DropoutPINN
from utils_uqmd.utils_uq_mlp import MLPPINN
from utils_uqmd.utils_uq_vi import VIBPINN
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# CP
from utils_uqmd.utils_uq_cp import CP

# Ensure reproducibility
import random, numpy as np, torch
seed = 12345
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Data Noise
data_noise = 0.05

# Define PDE
domain=((0.0, 1.0), (0.0, 1.0))
true_solution=lambda xy: torch.sin(math.pi * xy[..., 0:1]) * torch.sin(math.pi * xy[..., 1:2])
pde = Poisson2D(domain, true_solution)

# Generate training and testing data of the poisson function
(X_train, Y_train) = pde.data_generation(500, data_noise)
(X_test, Y_test) = pde.data_generation(200, data_noise)
(X_calibration, Y_calibration) = pde.data_generation(100, data_noise)


# Utility data point (Not training related)
(X_validation, Y_validation) = pde.data_generation(200, data_noise)
X_vis, Y_vis = pde.data_generation(800, data_noise)

# Number of colocation point
colloc_pt_num = 250

# Generating alphas to test
from utils_tools.utils_result_metrics import generating_alphas
alphas = generating_alphas()

alpha = 0.05

# --------------------------------------------
# Defining Testing
# --------------------------------------------
n_grid = 500
x = torch.linspace(0, 1, n_grid)
y = torch.linspace(0, 1, n_grid)
X, Y = torch.meshgrid(x, y, indexing='xy')
grid_test = torch.cat([X.reshape(-1, 1), Y.reshape(-1, 1)], dim=1)

# ────────────────────────────────────────────────────────────────────────────────
# 3. Visualization: Plot ground-truth & noisy data
# ────────────────────────────────────────────────────────────────────────────────
from utils_tools.utils_result_viz import plot_truth_and_samples_2D
from utils_tools.utils_tuning import save_plot
# Dense grid for nice plotting
def make_dense_grid(domain, N=200):
    x = np.linspace(domain[0][0], domain[0][1], N)
    y = np.linspace(domain[1][0], domain[1][1], N)
    X, Y = np.meshgrid(x, y)
    XY_grid = np.stack([X.ravel(), Y.ravel()], axis=-1)
    return XY_grid, X, Y

grid_plot, X_mesh, Y_mesh = make_dense_grid(domain, 200)
grid_test_torch = torch.tensor(grid_plot, dtype=torch.float32)
U_true_grid = true_solution(grid_test_torch).numpy().reshape(X_mesh.shape)


save_plot(
    plot_truth_and_samples_2D,
    save_dir="2D_Poisson", prefix="2d_poisson_training_data",
)(
    X_vis, Y_vis, grid_plot, U_true_grid, domain,
    title="2D Poisson Training Data"
)


Using device: cpu
Using device: cpu


/Users/yifanyu/miniconda3/envs/cp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<Figure size 640x480 with 0 Axes>

# Feature Distance Model

## Define Model

In [2]:
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# Base-Model Instance
model_args = {
    "pde_class":pde,
    "input_dim":2,
    "hidden_dims":[16, 32, 64, 64, 64, 32, 16], 
    "output_dim":1,
}

baseline_pred_kwargs = dict(n_samples=200)


# CP-Model

cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, 
    "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"feature", # Change base on if the baseline cp
    "k":20
}

baseline_testing_args = { 
    # "uqmodel":dist_pinn
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "heuristic_u":"feature",
    "n_samples":20, 
}

dist_feat_pinn = DistanceUQPINN(**model_args)

## Training Base Model

In [3]:
# Base Model
# Training
fit_args = dict(
    coloc_pt_num=colloc_pt_num, 
    X_train=X_train, 
    Y_train=Y_train
)
fit_kwargs_grid = dict(
    epochs=5000,
    λ_pde=1.0, λ_ic=10.0, λ_data=3.0,
    lr=1e-3, stop_schedule=20000
)
baseline_loss_dict = dist_feat_pinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = dist_feat_pinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=20, 
    heuristic_u="feature"
)
dist_feat_pinn_uncal_predset = dist_feat_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


ep     1 | L=1.11e+02 | lr=1.0e-03
ep   500 | L=6.58e-01 | lr=1.0e-03
ep  1000 | L=4.19e-01 | lr=1.0e-03
ep  1500 | L=3.74e-01 | lr=1.0e-03
ep  2000 | L=3.13e-01 | lr=1.0e-03
ep  2500 | L=1.42e-01 | lr=1.0e-03
ep  3000 | L=2.16e-01 | lr=1.0e-03
ep  3500 | L=1.08e-01 | lr=1.0e-03
ep  4000 | L=9.24e-02 | lr=1.0e-03
ep  4500 | L=6.54e-02 | lr=1.0e-03
ep  5000 | L=6.52e-02 | lr=1.0e-03


## Create CP Model

In [4]:
# CP Model
dist_feat_pinn_cp = CP(dist_feat_pinn)

# Predicting
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"feature",  # Change this based on cp
        "k":20
}

dist_feat_pinn_cp_cal_predset = dist_feat_pinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [5]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
dist_feat_pinn_df_uncal = dist_test_uncertainties(uqmodel=dist_feat_pinn, **baseline_testing_args)
dist_feat_pinndf_cal = cp_test_uncertainties(dist_feat_pinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_2D_comparison_with_coverage, plot_metrics_table
save_plot(
    plot_2D_comparison_with_coverage,
    save_dir="2D_Poisson", prefix="feature_distance",
    loss=baseline_data_loss
)(
    grid_test, dist_feat_pinn_uncal_predset, dist_feat_pinn_cp_cal_predset, 
    true_solution, 
    dist_feat_pinn_df_uncal, dist_feat_pinndf_cal,
    title="2D UQ", main_title="Feature Distance Model", 
    X_vis=X_train, Y_vis=Y_train
)



100%|██████████| 19/19 [00:01<00:00, 14.61it/s]


<Figure size 640x480 with 0 Axes>

In [6]:
save_plot(
    plot_metrics_table,
    save_dir="2D_Poisson", prefix="feature_distance_table",
    loss=baseline_data_loss
)(
    grid_test, dist_feat_pinn_uncal_predset, dist_feat_pinn_cp_cal_predset, 
    true_solution, 
    dist_feat_pinn_df_uncal, dist_feat_pinndf_cal,
    title="2D UQ", main_title="Feature Distance Model Metrics", 
    X_vis=X_train, Y_vis=Y_train
)

Uncal dev:0.07547618856742269
Cal dev:0.04809522965834254


<Figure size 640x480 with 0 Axes>

# Latent Distance Model

## Define Model

In [7]:
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# Base-Model Instance
model_args = {
    "pde_class":pde,
    "input_dim":2,
    "hidden_dims":[16, 32, 64, 64, 64, 32, 16], 
    "output_dim":1,
}


# CP-Model

cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, 
    "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"latent", # Change base on if the baseline cp
    "k":20
}

baseline_testing_args = { 
    # "uqmodel":dist_pinn
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "heuristic_u":"latent",
    "n_samples":20, 
}

dist_lat_pinn = DistanceUQPINN(**model_args)

## Training Base Model

In [8]:
# Base Model
# Training
fit_args = dict(
    coloc_pt_num=colloc_pt_num, 
    X_train=X_train, 
    Y_train=Y_train
)
fit_kwargs_grid = dict(
    epochs=5000,
    λ_pde=1.0, λ_ic=5.0, λ_data=1.0,
    lr=1e-3, stop_schedule=20000
)
baseline_loss_dict = dist_lat_pinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = dist_lat_pinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=20, 
    heuristic_u="latent"
)
dist_lat_pinn_uncal_predset = dist_lat_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


ep     1 | L=1.11e+02 | lr=1.0e-03
ep   500 | L=3.66e-01 | lr=1.0e-03
ep  1000 | L=1.97e-01 | lr=1.0e-03
ep  1500 | L=1.54e-01 | lr=1.0e-03
ep  2000 | L=1.25e-01 | lr=1.0e-03
ep  2500 | L=1.10e-01 | lr=1.0e-03
ep  3000 | L=9.77e-02 | lr=1.0e-03
ep  3500 | L=7.89e-02 | lr=1.0e-03
ep  4000 | L=4.49e-02 | lr=1.0e-03
ep  4500 | L=1.72e-02 | lr=1.0e-03
ep  5000 | L=5.57e-03 | lr=1.0e-03


## Create CP Model

In [9]:
# CP Model
dist_lat_pinn_cp = CP(dist_lat_pinn)

# Predicting
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"latent",  # Change this based on cp
        "k":20
}

dist_lat_pinn_cp_cal_predset = dist_lat_pinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [10]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
dist_lat_pinn_df_uncal = dist_test_uncertainties(uqmodel=dist_lat_pinn, **baseline_testing_args)
dist_lat_pinn_df_cal = cp_test_uncertainties(dist_lat_pinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_2D_comparison_with_coverage
save_plot(
    plot_2D_comparison_with_coverage,
    save_dir="2D_Poisson", prefix="latent_distance",
    loss=baseline_data_loss
)(
    grid_test, dist_lat_pinn_uncal_predset, dist_lat_pinn_cp_cal_predset, 
    true_solution, 
    dist_lat_pinn_df_uncal, dist_lat_pinn_df_cal,
    title="2D UQ", main_title="Latent Distance Model", 
    X_vis=X_train, Y_vis=Y_train
)


100%|██████████| 19/19 [00:01<00:00, 14.19it/s]


<Figure size 640x480 with 0 Axes>

In [11]:

save_plot(
    plot_metrics_table,
    save_dir="2D_Poisson", prefix="latent_distance_table",
    loss=baseline_data_loss
)(
    grid_test, dist_lat_pinn_uncal_predset, dist_lat_pinn_cp_cal_predset, 
    true_solution, 
    dist_lat_pinn_df_uncal, dist_lat_pinn_df_cal,
    title="2D UQ", main_title="Latent Distance Model Metrics", 
    X_vis=X_train, Y_vis=Y_train
)

Uncal dev:0.32619048193806693
Cal dev:0.019047622701951435


<Figure size 640x480 with 0 Axes>

# Dropout Model

## Define Model

In [12]:
from utils_uqmd.utils_uq_dropout import DropoutPINN

# Base Model Instance
model_args = {
    "pde_class":pde,
    "input_dim":2,
    "hidden_dims":[16, 32, 64, 64, 64, 32, 16], 
    "output_dim":1,
}
baseline_pred_kwargs = {  
    "n_samples":20 
}

# CP Model
cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, 
    "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"raw_std", # Change base on if the baseline cp
    "k":10
}

cp_pred_kwargs = {
    "X_train":X_train,  "Y_train":Y_train,
    "X_cal":X_calibration, "Y_cal":Y_calibration,
    "heuristic_u":"raw_std",
    "k":10
}

baseline_testing_args = { 
    # "uqmodel":do_pinn,   # Change this
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "n_samples":20
}

dropout_pinn = DropoutPINN(**model_args)

## Training Base Model

In [13]:
# Base Model
# Training
fit_args = dict(
    coloc_pt_num=colloc_pt_num, 
    X_train=X_train, 
    Y_train=Y_train
)
fit_kwargs_grid = dict(
    epochs=5000,
    λ_pde=1.0, λ_ic=5.0, λ_data=1.0,
    lr=1e-3, stop_schedule=20000
)
baseline_loss_dict = dropout_pinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = dropout_pinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=20
)
dropout_pinn_uncal_predset = dropout_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


ep     1 | L=1.11e+02 | data=4.23e-01 | pde=1.10e+02  ic=0.00e+00  bc=3.06e-02 | lr=1.00e-03
ep   500 | L=1.03e+01 | data=1.16e-01 | pde=8.73e+00  ic=0.00e+00  bc=1.48e-01 | lr=1.00e-03
ep  1000 | L=7.70e+00 | data=8.32e-02 | pde=6.29e+00  ic=0.00e+00  bc=1.32e-01 | lr=1.00e-03
ep  1500 | L=5.70e+00 | data=9.41e-02 | pde=4.44e+00  ic=0.00e+00  bc=1.17e-01 | lr=1.00e-03
ep  2000 | L=6.38e+00 | data=7.84e-02 | pde=5.14e+00  ic=0.00e+00  bc=1.16e-01 | lr=1.00e-03
ep  2500 | L=4.75e+00 | data=8.25e-02 | pde=3.52e+00  ic=0.00e+00  bc=1.15e-01 | lr=1.00e-03
ep  3000 | L=4.33e+00 | data=7.81e-02 | pde=3.15e+00  ic=0.00e+00  bc=1.10e-01 | lr=1.00e-03
ep  3500 | L=4.22e+00 | data=6.68e-02 | pde=3.15e+00  ic=0.00e+00  bc=1.00e-01 | lr=1.00e-03
ep  4000 | L=3.96e+00 | data=7.55e-02 | pde=2.92e+00  ic=0.00e+00  bc=9.70e-02 | lr=1.00e-03
ep  4500 | L=3.75e+00 | data=6.34e-02 | pde=2.80e+00  ic=0.00e+00  bc=8.84e-02 | lr=1.00e-03
ep  5000 | L=3.30e+00 | data=6.99e-02 | pde=2.34e+00  ic=0.00e+00  bc=

/Users/yifanyu/miniconda3/envs/cp/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


## Create CP Model

In [14]:
# CP Model
dropout_pinn_cp = CP(dropout_pinn)

# Predicting
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"raw_std",  # Change this based on cp
        "k":10
}

dropout_pinn_cp_cal_predset = dropout_pinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [15]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
dropout_pinn_df_uncal = do_test_uncertainties(uqmodel=dropout_pinn, **baseline_testing_args)
dropout_pinn_df_cal = cp_test_uncertainties(dropout_pinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_2D_comparison_with_coverage
save_plot(
    plot_2D_comparison_with_coverage,
    save_dir="2D_Poisson", prefix="dropout",
    loss=baseline_data_loss
)(
    grid_test, dropout_pinn_uncal_predset, dropout_pinn_cp_cal_predset, 
    true_solution, 
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
    title="2D UQ", main_title="Dropout Model", 
    X_vis=X_train, Y_vis=Y_train
)




100%|██████████| 19/19 [00:03<00:00,  6.14it/s]


<Figure size 640x480 with 0 Axes>

In [16]:
save_plot(
    plot_metrics_table,
    save_dir="2D_Poisson", prefix="dropout_table",
    loss=baseline_data_loss
)(
    grid_test, dropout_pinn_uncal_predset, dropout_pinn_cp_cal_predset, 
    true_solution, 
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
    title="2D UQ", main_title="Dropout Model Metrics", 
    X_vis=X_train, Y_vis=Y_train
)

Uncal dev:0.26428570899934994
Cal dev:0.047380946044410975


<Figure size 640x480 with 0 Axes>

# VI Model

## Define Model

In [17]:
from utils_uqmd.utils_uq_vi import VIBPINN

# Base Model Instance
model_args = {
    "pde_class":pde,
    "input_dim":2,
    "hidden_dims":[16, 32, 64, 64, 64, 32, 16], 
    "output_dim":1,
    "mu_std" : 0.01, "rho" : -5, "prior_std" : 1.0, 
    "init_data_noise" : 1.0, "learn_data_noise" : False, 
    "act_func" : nn.Tanh()
}

baseline_testing_args = { 
    # "uqmodel":vi_model, 
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test
}

vi_bpinn = VIBPINN(**model_args)

## Training Base Model

In [18]:
# Base Model
# Training
fit_args = {
    "coloc_pt_num":colloc_pt_num,
    "X_train":X_train, 
    "Y_train":Y_train
}
fit_kwargs_grid = {
    "epochs":30000,
    "λ_pde":5.0, "λ_bc":5.0, "λ_elbo":1.0,  
    "lr":3e-3,
    "stop_schedule":20000
}
baseline_loss_dict = vi_bpinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = vi_bpinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=5000
)
vi_bpinn_uncal_predset = vi_bpinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


ep     0 | L=6.06e+04 | elbo=1.47e+02 | pde=1.21e+04  ic=0.00e+00  bc=2.60e-06 | lr=3.00e-03 | learned noise_std=1.000e+00
ep     1 | L=6.06e+04 | elbo=1.44e+02 | pde=1.21e+04  ic=0.00e+00  bc=4.67e-05 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   300 | L=3.78e+03 | elbo=1.05e+03 | pde=5.39e+02  ic=0.00e+00  bc=6.24e+00 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   600 | L=2.84e+03 | elbo=7.89e+02 | pde=4.05e+02  ic=0.00e+00  bc=5.14e+00 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   900 | L=2.50e+03 | elbo=6.97e+02 | pde=3.57e+02  ic=0.00e+00  bc=4.73e+00 | lr=3.00e-03 | learned noise_std=1.000e+00
ep  1200 | L=2.42e+03 | elbo=7.06e+02 | pde=3.37e+02  ic=0.00e+00  bc=4.70e+00 | lr=3.00e-03 | learned noise_std=1.000e+00
ep  1500 | L=2.41e+03 | elbo=7.95e+02 | pde=3.17e+02  ic=0.00e+00  bc=5.16e+00 | lr=3.00e-03 | learned noise_std=1.000e+00
ep  1800 | L=2.40e+03 | elbo=7.32e+02 | pde=3.28e+02  ic=0.00e+00  bc=4.84e+00 | lr=3.00e-03 | learned noise_std=1.000e+00
ep  2100 | L=2.2

## Create CP Model

In [19]:
# CP Model
vi_bpinn_cp = CP(vi_bpinn)

# CP
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"raw_std",              
}

cp_testing_args = {
        "alphas":alphas, 
        "X_test":X_test, "Y_test":Y_test, 
        "X_cal":X_calibration, "Y_cal":Y_calibration, "X_train":X_train, "Y_train":Y_train, 
        "heuristic_u":"raw_std", # Change base on if the baseline model has its original uq band
        "k":10
}

vi_bpinn_cp_cal_predset = vi_bpinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [20]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
vi_bpinn_df_uncal = vi_test_uncertainties(uqmodel=vi_bpinn, **baseline_testing_args)
vi_bpinn_df_cal = cp_test_uncertainties(vi_bpinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_2D_comparison_with_coverage
save_plot(
    plot_2D_comparison_with_coverage,
    save_dir="2D_Poisson", prefix="vi_bpinn",
    loss=baseline_data_loss
)(
    grid_test, vi_bpinn_uncal_predset, vi_bpinn_cp_cal_predset, 
    true_solution, 
    vi_bpinn_df_uncal, vi_bpinn_df_cal,
    title="2D UQ", main_title="VI Model", 
    X_vis=X_train, Y_vis=Y_train
)



100%|██████████| 19/19 [08:00<00:00, 25.27s/it]


<Figure size 640x480 with 0 Axes>

In [21]:

save_plot(
    plot_metrics_table,
    save_dir="2D_Poisson", prefix="vi_bpinn_table",
    loss=baseline_data_loss
)(
    grid_test, vi_bpinn_uncal_predset, vi_bpinn_cp_cal_predset, 
    true_solution, 
    vi_bpinn_df_uncal, vi_bpinn_df_cal,
    title="2D UQ", main_title="VI Model Metrics", 
    X_vis=X_train, Y_vis=Y_train
)

Uncal dev:0.0990476246391024
Cal dev:0.016428574032726743


<Figure size 640x480 with 0 Axes>

# HMC Model

## Define Model

In [22]:
from utils_uqmd.utils_uq_hmc import HMCBPINN

# Base Model Instance
model_args = dict(
    pde_class=pde,input_dim=2, hidden_dims=[16, 32, 64, 64, 64, 32, 16],
    output_dim=1, act_func=nn.Tanh, prior_std=1.0,
    step_size=1e-3, leapfrog_steps=5
)

baseline_testing_args = {
    # "uqmodel": hmc_model,
    "alphas": alphas,
    "X_test": X_test,
    "Y_test": Y_test,
    "n_samples": 5000
}



hmc_bpinn = HMCBPINN(**model_args)

## Training Base Model

In [23]:
# Base Model
# Training
fit_args = {
    "coloc_pt_num":colloc_pt_num,
    "X_train":X_train, 
    "Y_train":Y_train
}
fit_kwargs_grid = {
    "λ_pde": 3.0,
    "λ_bc": 5.0,
    "λ_data": 1.0,
    "epochs": 5000,
    "lr":1e-3,
    "hmc_samples": 12000,
    "brun_in":5000,
    "step_size": 5e-4,
    "leapfrog_steps": 14,
    "lr_decay_step":2000
}
baseline_loss_dict = hmc_bpinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = hmc_bpinn.data_loss(X_validation, Y_validation)


# Predict 
baseline_pred_kwargs = { "n_samples": 5000 }
hmc_bpinn_uncal_predset = hmc_bpinn.predict(
            alpha=alpha, X_test=grid_test,
            **baseline_pred_kwargs
        )

MAP:  10%|█         | 509/5000 [00:05<00:44, 100.57it/s, loss=1.10e+02]

[MAP] epoch    500 −logPost=1.104e+02  Data=1.47e-01  PDE=1.10e+02  IC=0.00e+00  BC=2.53e-02


MAP:  20%|██        | 1019/5000 [00:10<00:40, 98.46it/s, loss=1.10e+02]

[MAP] epoch   1000 −logPost=1.102e+02  Data=1.44e-01  PDE=1.10e+02  IC=0.00e+00  BC=2.69e-02


MAP:  30%|███       | 1517/5000 [00:15<00:34, 100.17it/s, loss=1.10e+02]

...

HMC:  92%|█████████▏| 11001/12000 [29:53<01:54,  8.75it/s, acc=0.72]

[HMC] iter  11000  acc-rate=0.72


HMC:  96%|█████████▌| 11501/12000 [30:51<00:56,  8.90it/s, acc=0.72]

[HMC] iter  11500  acc-rate=0.72


[HMC] iter  12000  acc-rate=0.72
Finished HMC: avg acceptance 0.721
Kept 11500 posterior samples


## Create CP Model

In [ ]:
# CP Model
hmc_bpinn_cp = CP(hmc_bpinn)

# CP
cp_pred_kwargs = { 
    "X_train": X_train, "Y_train": Y_train,
    "X_cal": X_calibration, "Y_cal": Y_calibration,
    "heuristic_u": "raw_std", "k": 10
}

cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"raw_std", # Change base on if the baseline model has its original uq band
    "k":10
}

hmc_bpinn_cp_cal_predset = hmc_bpinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [ ]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import hmc_test_uncertainties, cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
hmc_bpinn_df_uncal = hmc_test_uncertainties(uqmodel=hmc_bpinn, **baseline_testing_args)
hmc_bpinn_df_cal = cp_test_uncertainties(hmc_bpinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_2D_comparison_with_coverage
save_plot(
    plot_2D_comparison_with_coverage,
    save_dir="2D_Poisson", prefix="hmc_bpinn",
    loss=baseline_data_loss
)(
    grid_test, hmc_bpinn_uncal_predset, hmc_bpinn_cp_cal_predset, 
    true_solution, 
    hmc_bpinn_df_uncal, hmc_bpinn_df_cal,
    title="2D UQ", main_title="HMC Model", 
    X_vis=X_train, Y_vis=Y_train
)



  0%|          | 0/19 [00:00<?, ?it/s]/Users/yifanyu/miniconda3/envs/cp/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
100%|██████████| 19/19 [01:19<00:00,  4.18s/it]


<Figure size 640x480 with 0 Axes>

In [ ]:
save_plot(
    plot_metrics_table,
    save_dir="2D_Poisson", prefix="hmc_bpinn_table",
    loss=baseline_data_loss
)(
    grid_test, hmc_bpinn_uncal_predset, hmc_bpinn_cp_cal_predset, 
    true_solution, 
    hmc_bpinn_df_uncal, hmc_bpinn_df_cal,
    title="2D UQ", main_title="HMC Model Metrics", 
    X_vis=X_train, Y_vis=Y_train
)

<Figure size 640x480 with 0 Axes>